# Functional Programming in Python
## David Mertz
### dmertz@continuum.io
### 2016-04-22

# Table of Contents
* [Advanced Python](#Advanced-Python)
* [Learning Objectives:](#Learning-Objectives:)
	* [Magic Methods](#Magic-Methods)
	* [Comparison Methods](#Comparison-Methods)
	* [Arithmetic Methods](#Arithmetic-Methods)
* [Exercise (create a magic object)](#Exercise-%28create-a-magic-object%29)

# Advanced Python

# Learning Objectives:

* Magic methods and overloading operators
* The `__new__()` method, and its difference from `__init__()`
* Defining various operators or other syntax sugar, e.g. `__mul__()`, `__call__()`, `__lt__()`
* Unicode subtleties
* Generators and coroutines
* Metaclasses
* Decorators
* Context managers
* C3 MRO linearization
* Longer discussion of complexity classes of standard containers
* Functional programming techniques
* Properties and accessors
* Closures and function factories
* import hooks

Highlighting some standard library modules such as:
* asyncore
* multiprocessing
* subprocess
* threading
* ast and dis

## Magic Methods

Python objects have various methods that can be used.  Sometimes called "dunder" methods because of the double underscores surrounding the method name, these methods are used internally by Python to manipulate an object.
Note that the base object type in Python has generic versions of these methods defined.  Your object will inherit these methods when it inherits from ```object```.

Creating vs Initializing an object.
There are two methods that Python calls on every object that is created.
These methods are ```__new__()``` and ```__init__()```.  In most cases, defining a ```__new__``` method is not necessary.
```__new__()``` is called the creation of a new object.  By the time ```__init__()``` is called, the object already exists and only needs to be initialized.  If you're coming from a language like C/C++, ```__init__()``` != object constructor; ```__new__()``` == object constructor.

In [ ]:
# Create an instance whose type is determined at runtime
import numbers
import decimal
numbers.Real.register(decimal.Decimal)
class RealNumber(object):
    def __new__(cls, val, style=float):
        print("Creating the object as type", style, "using value", repr(val))
        # Here is where we create the new object.
        try:
            instance = style.__new__(style, val)
        except (decimal.InvalidOperation, ValueError, TypeError) as err:
            print("... must evaluate expression to float first")
            instance = style.__new__(style, eval(val))
        if not isinstance(instance, numbers.Real):
            raise TypeError("Must create Real number type, not %s" % style)
        # We must return the new object, or our work was for naught.
        return instance

In [ ]:
from fractions import Fraction
from decimal import Decimal
x = RealNumber("1/3", style=Fraction)
y = RealNumber("1/3", Decimal)
z = RealNumber("1/3", float)
w = RealNumber("1/3", int)
x, y, z, w

In [ ]:
RealNumber("1/3")
RealNumber(1/3);

In [ ]:
s = RealNumber("1/3", str)

In [ ]:
c = RealNumber("1+1j", complex)

## Comparison Methods

Sometimes, we may wish to customize how to compare object with other objects.  We can define what different comparison operators mean.  These comparison methods should return a boolean (True/False) value.  All of these methods take one argument, a reference to the object to use for comparing.

| Method  | Operator  |
|---|---|
| ```__lt__```  | <  |
| ```__eq__```  | ==  |
| ```__le__```  | <=  |
| ```__gt__```  | > 
| ```__ge__``` | >= |
| ```__ne__``` | != |

Lets create a class knows how to compare itself to sequences by comparing itself to each element of the sequence

In [ ]:
from collections.abc import Iterable

class Number(object):
    def __init__(self, num):
        self.num = num
        
    def __lt__(self, other):
        # Can check whether `other` is iterable
        if isinstance(other, Iterable):
            return all(self.num < i for i in other)
        return self.num < other
    
    def __eq__(self, other):
        # ...Or use Python duck-typing 
        # (i.e. it's easier to get forgiveness than permission) 
        try:
            return all(self.num == i for i in other)
        except TypeError as err:
            return self.num == other

In [ ]:
x = Number(3.14)
print(x < 2.16)
print(x < (.16, 3.14, 9.2))
print(x < (20, 30, 40))
print(x == [x, x, x, x, 3.14, 3.14])

In [ ]:
# Even though we have an interesting "compare to all elements,"
# this doesn't assure that comparisons always succeed
print(x == ['a', 'b', 'c'])
try:
    print(x < ['a', 'b', 'c'])
except TypeError as e:
    print("Cannot compare object with elements")

Notice that *only* less-than was implemented in `Number`, so other inequality comparisons will fail.

In [ ]:
expressions = ['x >= 2.16', 'x <= (.16, 3.14, 9.2)', 'x < (1, 2)', 'x > (20, 30, 40)']
x = Number(3.14)
for exp in expressions:
    try:
        print("%s: %s" % (eval(exp), exp))
    except TypeError as e:
        print("TypeError: %s" % exp)

We can remedy this by creating a total ordering from implemented comparisons.

In [ ]:
import functools
@functools.total_ordering
class BetterNumber(Number): 
    pass

x = BetterNumber(3.14)
for exp in expressions:
    try:
        print("%s: %s" % (eval(exp), exp))
    except TypeError as e:
        print("TypeError: %s" % exp)

In [ ]:
class SillyNumber(object):
    def __init__(self, num):
        self.num = num
    def __lt__(self, other):
        return True
    def __gt__(self, other):
        return True
        
x, y = SillyNumber(3.14), SillyNumber(2.71)

In [ ]:
x < y, x > y

In [ ]:
x < "Ham sandwich", x > "Ham sandwich"

When you call ```len()``` or ```str()``` on an object, Python really calls the ```__len__()``` and ```__str__()``` methods of the object.

In [ ]:
class EmptyThing(object):
    def __init__(self, *args):
        self.things = args
        number = "s" if len(args) > 1 else ""
        print("EmptyThing with %d thing%s" % (len(args), number))
        
    def __len__(self):
        # Must return an integer
        return 0  # never has positive length
    
    def __str__(self):
        # this method returns a string; it does not print anything.
        return "Things I have: " + str(self.things)

In [ ]:
x = EmptyThing(3, 5, 2, 1, 9, 0)
print(len(x)) # calls EmptyThing.__len__
print(str(x)) # calls EmptyThing.__str__
print(x)      # also calls EmptyThing.__str__


In [ ]:

# Calling .__len__() is one way of determining "truthiness" of object
print(bool(EmptyThing(3.14)), bool(Number(3.14)))

## Arithmetic Methods

We can also override what basic arithmetic operations mean to an object.  For instance, we can define what it means to add two instances of a class together with the `+` operator.

Python will look at an expression like `x+y` and attempts to resolve the addition operation as follows:

1. Try x and see if it can add the object y to itself.  This is called left addition, where the left-side object is responsible for carrying out the operation.
2. If that fails, try to see if y knows how to add the object x to itself.  This is right addition, where the right-side object is now responsible for performing the operation.
3. If that fails, return an error to the user

In [ ]:
# Let's implement a mathematical object called a ring.  
# We will implement the ring Z4 (math in the integers mod 4)
import numbers

class Z4(object):
    def __init__(self, a):
        if not isinstance(a, numbers.Integral):
            raise TypeError("Z4 ring only defined on Natural numbers")
        self.a = a % 4
        
    def __add__(self, other):
        # This is left addition for self + other
        if isinstance(other, type(self)):
            return Z4((self.a + other.a) % 4)
        return NotImplemented
        
    def __sub__(self, other):
        # This is for subtraction for self - other
        if isinstance(other, type(self)):
            return Z4((self.a - other.a) % 4)
        return NotImplemented
        
    def __truediv__(self, other):
        # This is for division for self / other
        if isinstance(other, type(self)):
            return Z4((self.a // other.a) % 4)
        return NotImplemented
        
    def __mul__(self, other):
        # This is for multiplication for self * other
        if isinstance(other, type(self)):
            return Z4((self.a * other.a) % 4)
        return NotImplemented
        
    def __str__(self):
        return str(self.a)
    
    def __repr__(self):
        return "Z4(%s)" % self.a

In [ ]:
v = Z4(2)
c = Z4(3)
print(v*c, v+c, v-c, v/c)

# Exercise (create a magic object)

Take a look at the ["special method names"](https://docs.python.org/3/reference/datamodel.html) in Python. Create one or more classes, and instances thereof, that respond to the Python syntax we've learned in *special* or surprising ways.  Try to think of something that is actually intuitive or useful in these behaviors.

In [ ]:
# An example of "interesting" special behavior
# A very small and terrible "DSL" (domain specific language)
class PeterPiper(object):
    def __init__(self, who="Peter Piper"):
        self.who = who
    def __call__(self):
        return "%s picked a peck of pickled peppers" % self.who
    def __floordiv__(self, other):
        self.who = other
    
class ShellCat(object):
    def __lt__(self, other):
        print(other)
        
david = PeterPiper('David')
cat = ShellCat()

cat < david()
david // "Bruce"
cat < david()

In [ ]:
# People who want Python to be Ada, C#, Haskell, or MATLAB can do this...
class MyInt(int):
    def __xor__(self, other):
        return self ** other
    
x = MyInt(5)
x ^ 3

In [ ]:
import sys, webbrowser
from time import sleep

class StringImpossible(str):
    def __str__(self):
        webbrowser.open('https://youtu.be/tGSUjuSBt1A')
        for name, x in globals().items():
            if x is self:
                break
        del globals()[name]
        sleep(10)      
        return "Your mission, should you care to accept it, %s ..." % self[:]
        
spy = StringImpossible("David")
print(spy)
for _ in range(40):
    sys.stdout.write('.')
    sleep(1)
print(spy)

In [ ]:
import continuum_style; continuum_style.style()